# Full Stack Semantic Search Web Application

In the previous modules, we've demonstrated both keyword and semantic search with OpenSearch. In this module, we will now create a search enabled application using a sage maker endpoint and a serverless web application.

By the end of this module, the architecture will look as follows:

![full stack semantic search](semantic_search_fullstack.jpg)

### 1.Import PyTorch and check version.

As in the previous modules, let's import PyTorch and confirm that have have the latest version of PyTorch. The version should already be 1.10.2 or higher.

In [ ]:
import torch
print(torch.__version__)

If the PyTorch version is 1.10.2 or higher, you can skip to step 2.

If the PyTorch version is not 1.10.2 or higher, we need to update PyTorch and restart the kernel.

In [ ]:
!pip install --upgrade torch

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

Let's recheck the version of Torch to ensure everything is up to date. The version should be 1.10.2 or higher.

In [ ]:
import torch
print(torch.__version__)

### 2. Initialize boto3

We will use boto3 to interact with other AWS services.

Note: You can ignore any PythonDeprecationWarning warnings.

In [ ]:
import boto3
import re
import time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')


### 3. Get Cloud Formation stack output variables

We also need to grab some key values from the infrastructure we provisioned using CloudFormation. To do this, we will list the outputs from the stack and store this in "outputs" to be used later.

#### Note : The following refers the stack by name. If you didn't use the default stack name, please update the value of "cloudformation_stack_name" to the Cloud Formation stack name you specified when you provisioned your environment.

You can ignore any "PythonDeprecationWarning" warnings.

In [ ]:
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "static-cloudformation-semantic-search"

outputs = get_cfn_outputs(cloudformation_stack_name)

bucket = outputs['s3BucketTraining']
aos_host = outputs['OpenSearchDomainEndpoint']

outputs

### 4. Save pre-trained BERT model to S3

First off, we will host a pretrained BERT model in a SageMaker Pytorch model server to generate 768x1 dimension fixed length sentence embedding from [sentence-transformers](https://github.com/UKPLab/sentence-transformers) using [HuggingFace Transformers](https://huggingface.co/sentence-transformers/distilbert-base-nli-stsb-mean-tokens). 

This SageMaker endpoint will be called by the application to generate vector data for the search query. First we'll get a pre-trained model and upload to S3

In [ ]:
#!pip install -q transformers
#!pip install -q boto3
#!pip install -q requests
#!pip install -q requests-aws4auth
#!pip install -q opensearch-py
#!pip install -q tqdm
#!pip install -q install transformers[torch]
#!pip install -U sentence-transformers rank_bm25
!pip install -q transformers
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertTokenizer, DistilBertModel
import os
from transformers import AutoTokenizer, AutoModel

model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
saved_model_dir = 'transformer'
os.makedirs(saved_model_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name) 

tokenizer.save_pretrained(saved_model_dir)
model.save_pretrained(saved_model_dir)

Create a SageMaker session and get the execution role to be used later.

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


Unpack the model

In [ ]:
!cd transformer && tar czvf ../model.tar.gz *

And finally upload the model to S3.

In [ ]:
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='sentence-transformers-model')
inputs

### 5. Create PyTorch Model Object

Next we need to create a PyTorchModel object. The deploy() method on the model object creates an endpoint which serves prediction requests in real-time. If the instance_type is set to a SageMaker instance type (e.g. ml.m5.large) then the model will be deployed on SageMaker. If the instance_type parameter is set to local then it will be deployed locally as a Docker container and ready for testing locally.

We need to create a Predictor class to accept TEXT as input and output JSON. The default behaviour is to accept a numpy array.

In [ ]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

### 6. Deploy the BERT model to SageMaker Endpoint
Now that we have the predictor class, let's deploy a SageMaker endpoint for our application to invoke.

#### Note: This process will take serveral minutes to complete.

You can ignore the "content_type is a no-op in sagemaker>=2" warning.

In [ ]:
pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code',
                             py_version = 'py38', 
                             framework_version = '1.10.2',
                             predictor_cls=StringPredictor)

predictor = pytorch_model.deploy(instance_type='ml.m5d.large', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'semantic-search-model-{int(time.time())}')

### 7. Test the SageMaker Endpoint.

Now that the endpoint is created, let's quickly test it out.

In [ ]:
import json
original_payload = 'Does this work with xbox?'
features = predictor.predict(original_payload)
vector_data = json.loads(features)

vector_data

----

# Deploying a full-stack semantic search application

We are now ready to build a real-world full-stack ML-powered web app. The Serverless Application Model (SAM) template we create below will deploy an Amazon API Gateway and AWS Lambda function. The Lambda function runs your code in response to HTTP requests that are sent to the API Gateway.

### 7. Build lambda zip file

First, we need to package our lambda function for deployment.

In [ ]:
%cd backend/lambda
!sh build-lambda.sh
!unzip -l lambda.zip
%cd /home/ec2-user/SageMaker/semantic-search

Upload the packaged Lambda zip file to S3

In [ ]:
s3_resource.Object(bucket, 'lambda/lambda.zip').upload_file('./backend/lambda/lambda.zip',ExtraArgs={'ACL':'public-read'})
lambda_zip_url = f'{bucket}'
print("lambada zip file url: " + lambda_zip_url)

### 8. Deploy a CloudFormation stack to create API Gateway and Lambda function

Next, we'll create a link to deploy a CloudFormation stack for our SAM application. Execute the following, then click on the link generated - this will open a new tab. Mark all the checkboxes at the end of the form and click Create Stack.

In [ ]:
s3_resource.Object(bucket, 'backend/template.yaml').upload_file('./backend/template.yaml', ExtraArgs={'ACL':'public-read'})


sam_template_url = f'https://{bucket}.s3.amazonaws.com/backend/template.yaml'
print("cloudformation template url:" + sam_template_url)


# Generate the CloudFormation Quick Create Link

print("Click the URL below to create the backend API for NLU search:\n")
print((
    'https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/review'
    f'?templateURL={sam_template_url}'
    '&stackName=semantic-search-api'
    f'&param_BucketName={outputs["s3BucketTraining"]}'
    f'&param_DomainName={outputs["OpenSearchDomainName"]}'
    f'&param_ElasticSearchURL={outputs["OpenSearchDomainEndpoint"]}'
    f'&param_SagemakerEndpoint={predictor.endpoint}'
    f'&param_LambdaZipFile={lambda_zip_url}'
))

### 9. Wait for the CloudFormation stack to complete.
Before proceeding further, wait for the CloudFormation stack to become complete. The status should change to "CREATE_COMPLETE".

### 10. Update the front end config

Next, we need to update the config of the front end with the API values.

In [ ]:
import json
api_endpoint = get_cfn_outputs('semantic-search-api')['TextSimilarityApi']

with open('./frontend/src/config/config.json', 'w') as outfile:
    json.dump({'apiEndpoint': api_endpoint}, outfile)

### 11. Deploy frontend services

Now that we've updated the configuration, we need to build and deploy our front end.

In [ ]:
# add NPM to the path so we can assemble the web frontend from our notebook code

from os import environ

npm_path = ':/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin'

if npm_path not in environ['PATH']:
    ADD_NPM_PATH = environ['PATH']
    ADD_NPM_PATH = ADD_NPM_PATH + npm_path
else:
    ADD_NPM_PATH = environ['PATH']
    
%set_env PATH=$ADD_NPM_PATH

In [ ]:
%cd ./frontend/

!npm install

In [ ]:
!npm run-script build

In [ ]:
hosting_bucket = f"s3://{outputs['s3BucketHostingBucketName']}"

!aws s3 sync ./build/ $hosting_bucket --acl public-read

### 12. Browse to the application

Now that the application is deployed, let's browse to the front end and test it out. Execute the following and click on the link generated.

In [ ]:
print('Click the URL below:\n')
print(outputs['S3BucketSecureURL'] + '/index.html')

You can search the question, for example "does this work with xbox?", compare the search result. you will see the difference between keyword search and semantic search.

![full stack semantic search](full-stack-semantic-search-ui.jpg)

In keyword search, some questions like "Does this work for a switch?", "does this work with pc" which include "does this work" are searched however the meaning is totally different with query.

In semantic search, some questions like "Do I need to buy anything extra to used in xbox one s controller?", "How do these headphones connect to the Xbox360 controller?" are searched. The meaning is very close to the query.
![full stack semantic search](full-stack-semantic-search-ui-2.jpg)

## Cleanup

If you are running this workshop from a Workshop Studio AWS account provided to you by AWS, resources will be automatically cleaned up at the end of the event. However, if you are runnign this workshop in your own account, make sure that you stop the notebook instance, delete the Amazon SageMaker endpoint and delete the Elasticsearch domain to prevent any additional charges.

In [ ]:
# Delete the endpoint
predictor.delete_endpoint()

# Empty S3 Contents
training_bucket_resource = s3_resource.Bucket(bucket)
training_bucket_resource.objects.all().delete()

hosting_bucket_resource = s3_resource.Bucket(outputs['s3BucketHostingBucketName'])
hosting_bucket_resource.objects.all().delete()